## Comparison of GloGEM, PyGEM, and OGGM RGI 16 Runoff Outputs 

This notebook imports and processes GloGEM, PyGEM, and OGGM RGI-01 outputs. Summing glacial runoff by basin, we produce a plot that compares the three models' projected runoff values for each basin by SSP. 

Last Updated: 7 July 2023 | FFW

### Aggregation of GloGEM runoff:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import date
import collections
import datetime
import os
import xarray as xr

## Generic the filepath to the main data folder
fpath0 = '/Users/finnwimberly/Library/CloudStorage/GoogleDrive-fwimberly@middlebury.edu/My Drive/Lizz Research Stuff/Runoff-intercomparison/GloGEM-output/RGI18-NewZealand/files/'  

#All of the climate models used
modelnames = ['BCC-CSM2-MR','CAMS-CSM1-0','CESM2','CESM2-WACCM','EC-Earth3','EC-Earth3-Veg','FGOALS-f3-L','GFDL-ESM4',
              'INM-CM4-8','INM-CM5-0','MPI-ESM1-2-HR','MRI-ESM2-0', 'NorESM2-MM']

SSPpaths = ['ssp126','ssp245','ssp370','ssp585']   #Specifiying the SSP
#SSPs = ['ssp119','ssp126','ssp245','ssp370','ssp585'] #Use a different path as we have all 5 ssps for volume

In [ ]:
all_discharges = [[] for _ in SSPpaths]

for s, SSPpath in enumerate(SSPpaths):
    model_discharges = []
    for modelname in modelnames:
        temp_df = pd.read_csv(fpath0 + modelname + '/' + SSPpaths[s]  + '/' + 'newzealand_Discharge_r1.dat', sep='\s+', header=None, skiprows=1, index_col=0)
        model_discharges.append(temp_df)
    all_discharges[s] = model_discharges

In [ ]:
all_areas = [[] for _ in SSPpaths]

for s, SSPpath in enumerate(SSPpaths):
    model_areas = []
    for modelname in modelnames:
        temp_df = pd.read_csv(fpath0 + modelname  + '/' + SSPpaths[s]  + '/' + 'newzealand_Area_r1.dat', sep='\s+', index_col="ID")
        model_areas.append(temp_df)
    all_areas[s] = model_areas

In [ ]:
# Create new index using pandas date_range function
start_date = datetime.date(1980, 1, 1)
end_date = datetime.date(2100, 12, 1)
new_indices = pd.date_range(start_date, end_date, freq='MS').strftime('%Y-%m').tolist()

# Apply new index and datetime conversion
for s, SSPpath_discharges in enumerate(all_discharges):
    for m, discharge_df in enumerate(SSPpath_discharges):
        all_discharges[s][m].columns = new_indices
        all_discharges[s][m].columns = pd.to_datetime(new_indices)

In [ ]:
#expanding area dataset to match year-month dimension
for s in range(len(SSPpaths)):
    for i in range(len(all_areas[s])):
        all_areas[s][i] = all_areas[s][i][all_areas[s][i].columns.repeat(12)]
        
for s, areas in enumerate(all_areas):
    for i, area in enumerate(areas):
        all_areas[s][i].columns = new_indices
        all_areas[s][i].columns = pd.to_datetime(new_indices)

In [ ]:
# We will use our initial area to compute runoff so we fill entire dfs with element 0
# We only use one SSP because the initial areas are all the same -- we save time without looping through all
# We also convert km^2 to m^2

initial_areas = [pd.DataFrame(df.iloc[:, 0].values.repeat(df.shape[1]).reshape(df.shape), index=df.index, columns=df.columns).mul(1e6) for df in all_areas[0]]

In [ ]:
runoff = {s: {m: None for m in modelnames} for s in SSPpaths} # create nested dictionary indexed by model name and ssp
n=0
for s in SSPpaths:
    i=0
    for m in modelnames:
        runoff[s][m] = pd.concat([initial_areas[i] * all_discharges[n][i]], axis=1)
        i+=1
    n+=1

In [ ]:
import itertools
annualrunoff = {s: {m: None for m in modelnames} for s in SSPpaths}
for s, m in itertools.product(SSPpaths, modelnames):
    annualrunoff[s][m] = runoff[s][m].transpose().resample('A').sum() * 1e-9  #m^3 to km^3

In [ ]:
import json
def select_glaciers_json(basin='all'):
    '''
    Select glaciers within a basin by MRBID from a json-file,
    which is stored in the data directory.

    Args:
    -----
    basin: str
        String of MRBID or 'all'.

    Returns:
    --------
    If basin is 'all' a list of all relevant glaciers is returned, for
    initiating glacier simulations. If basin is a MRBID the list of glaciers
    within that basin is returned.
    
    Copy of a function written by Erik Holmgren (2022) in holmgren_gha.utils
    '''

    # fpath = './data/rgi_ids_per_basin.json'
    fpath = '/Users/finnwimberly/Library/CloudStorage/GoogleDrive-fwimberly@middlebury.edu/My Drive/Lizz Research Stuff/rgi_ids_per_basin.json'  
    with open(fpath) as f:
        basin_dict = json.load(f)

    if basin.lower() != 'all':
        glacier_list = basin_dict[basin]
    else:
        glacier_list = list(itertools.chain.from_iterable(basin_dict.values()))

    return glacier_list

In [ ]:
def sum_basin(basin_RGI_list, runoff_data):
    # Create new list to match our RGI formatting
    new_basin_list = [int(str(x)[-4:]) for x in basin_RGI_list]
    runoff_data = runoff_data.transpose()
    
    #TODO: create list of glaciers within a basin that are not included in GloGEM output
    # Filter new_basin_list to keep only the indexes present in the DataFrame
    new_basin_list = [x for x in new_basin_list if x in runoff_data.index]
    
    # Extract glaciers contained in the list from original df and create a new df
    new_df = runoff_data.loc[new_basin_list].copy()
    
    # Sum the values of the glaciers within the basin
    summed_basin_runoff = new_df.sum()
    #print(summed_basin_runoff)
    
    return summed_basin_runoff

In [ ]:
#Generating the aggregated basin data
Alpine_basins = {'CLUTHA':'5406'}

basins = ['CLUTHA']

modelnames_glo = ['BCC-CSM2-MR','CAMS-CSM1-0','CESM2','CESM2-WACCM','EC-Earth3','EC-Earth3-Veg','FGOALS-f3-L','GFDL-ESM4',
              'INM-CM4-8','INM-CM5-0','MPI-ESM1-2-HR','MRI-ESM2-0', 'NorESM2-MM']

scenarios = ['ssp126','ssp245','ssp370','ssp585']

basin_sums_glo = {}
basin_sums_monthly_glo = {}
for s, SSP in enumerate(scenarios):
    basin_sums_glo[SSP] = {}
    basin_sums_monthly_glo[SSP] = {}
    for b, basin in enumerate(basins):
        basin_sums_glo[SSP][basin] = {}
        basin_sums_monthly_glo[SSP][basin] = {}
        for m, model in enumerate(modelnames_glo):
            basin_sums_glo[SSP][basin][model] = sum_basin(select_glaciers_json(Alpine_basins[basin]), annualrunoff[SSP][model]) 
            basin_sums_monthly_glo[SSP][basin][model] = sum_basin(select_glaciers_json(Alpine_basins[basin]), runoff[SSP][model].transpose()*1e-9) 

In [ ]:
#To calculate multi GCM means and Quartiles we convert to df then calculate across first axis (GCMs)
GCM_mean_glo = {}
GCM_q1_glo = {}
GCM_q3_glo = {}
for s, SSP in enumerate(SSPpaths):
    GCM_mean_glo[SSP] = {}
    GCM_q1_glo[SSP] = {}
    GCM_q3_glo[SSP] = {}
    for b, basin in enumerate(basins):
        GCM_mean_glo[SSP][basin] = pd.DataFrame(basin_sums_glo[SSP][basin]).mean(axis=1)
        GCM_q1_glo[SSP][basin] = pd.DataFrame(basin_sums_glo[SSP][basin]).quantile(q=0.25, axis=1)
        GCM_q3_glo[SSP][basin] = pd.DataFrame(basin_sums_glo[SSP][basin]).quantile(q=0.75, axis=1)

### Aggregation of PyGEM runoff:

In [ ]:
import xarray as xr

#All of the climate models used
modelnames_py = ['BCC-CSM2-MR','CESM2','CESM2-WACCM','EC-Earth3','EC-Earth3-Veg','FGOALS-f3-L','GFDL-ESM4',
              'INM-CM4-8','INM-CM5-0','MPI-ESM1-2-HR','MRI-ESM2-0', 'NorESM2-MM']

SSPs = ['ssp126','ssp245','ssp370','ssp585'] #List of all SSPs in PyGEM

Alpine_basins = {'CLUTHA':'5406'}

basins = ['CLUTHA']

#Generic filepath to navigate to Drive folder 
fpathPy = '/Users/finnwimberly/Library/CloudStorage/GoogleDrive-fwimberly@middlebury.edu/My Drive/Lizz Research Stuff/Runoff-intercomparison/PyGEM/18'

In [ ]:
basin_gls = {}
for basin, code in Alpine_basins.items():
    basin_gls[basin] = select_glaciers_json(code)

In [ ]:
#Importing all runoff data, taking annual sum, and converting m^3 to km^3
import glob   #use glob to group files by filename similarities (in this case, SSP)

rf_ds = {}
annual_rf_ds = {}
for s, SSP in enumerate(SSPs):
    fpath1 = '/R18_runoff_monthly_c2_ba1_1set_2000_2100-{}'.format(SSP)
    file_pattern = f'{fpathPy + fpath1}*.nc'
    file_list = glob.glob(file_pattern)
    #print(file_list)
    
    datasets = []  # Create an empty list for each SSP
    if file_list:
        for file in file_list:
            with xr.open_dataset(file) as ds:
                ds = ds.glac_runoff_monthly.load()
                datasets.append(ds)
    
        combined_ds = xr.concat(datasets, dim='glacier')  # Concatenate the datasets
        rf_ds[SSP] = combined_ds
        annual_rf_ds[SSP] = rf_ds[SSP].resample(time='A').sum() * 1e-9

In [ ]:
# Sorting into basins
basin_datasets = {}
basin_ds_monthly = {}
for basin, glacier_list in basin_gls.items():
    ## loop over them all, drop the irrelevant IDs, and concatenate the result
    basin_datasets[basin] = {}
    basin_ds_monthly[basin] = {}
    for s, SSP in enumerate(SSPs):
        ds_list = []
        ds_list_monthly = []
        try:
            ds_filtered = annual_rf_ds[SSP].where(annual_rf_ds[SSP].RGIId.isin(glacier_list), drop=True)
            ds_filtered_monthly = rf_ds[SSP].where(rf_ds[SSP].RGIId.isin(glacier_list), drop=True)
            #print(ds_filtered)
            ds_list.append(ds_filtered)
            ds_list_monthly.append(ds_filtered_monthly)
        except ValueError: ## happens if there are no glaciers from this batch in the selected region
            continue
        basin_datasets[basin][SSP] = xr.concat(ds_list, dim='glacier')
        basin_ds_monthly[basin][SSP] = xr.concat(ds_list_monthly, dim='glacier') 

In [ ]:
#Flipping indexing (to match other models) and summing basins
basin_sums_py = {}
basin_sums_monthly_py = {}
for s, SSP in enumerate(SSPs):        
    basin_sums_py[SSP] = {}
    basin_sums_monthly_py[SSP] = {}
    for basin, glacier_list in basin_gls.items():
        basin_sums_py[SSP][basin] = basin_datasets[basin][SSP].sum(dim='glacier')
        basin_sums_monthly_py[SSP][basin] = basin_ds_monthly[basin][SSP].sum(dim='glacier')*1e-9

In [ ]:
#Compute multi GCM means and quartiles
GCM_mean_py = {}
GCM_q1_py = {}
GCM_q3_py = {}
for s, SSP in enumerate(SSPs):
    GCM_mean_py[SSP] = {}
    GCM_q1_py[SSP] = {}
    GCM_q3_py[SSP] = {}
    for basin in basins:
        GCM_mean_py[SSP][basin] = basin_sums_py[SSP][basin].mean(dim = 'model')
        GCM_q1_py[SSP][basin] = basin_sums_py[SSP][basin].quantile(q = 0.25, dim = 'model')
        GCM_q3_py[SSP][basin] = basin_sums_py[SSP][basin].quantile(q = 0.75, dim = 'model')

### Aggregation of OGGM Runoff

In [ ]:
#All of the climate models used
modelnames_OG = ['BCC-CSM2-MR', 'CAMS-CSM1-0', 'CESM2', 'CESM2-WACCM', 'CMCC-CM2-SR5','EC-Earth3', 
                'EC-Earth3-Veg', 'FGOALS-f3-L', 'GFDL-ESM4', 'INM-CM4-8','INM-CM5-0', 
                 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'NorESM2-MM', 'TaiESM1']

Alpine_basins = {'CLUTHA':'5406'}

basins = ['CLUTHA']

SSPs = ['ssp126','ssp245','ssp370','ssp585']

# CMCC-CM2-SR5 & TaiESM1 only hold values for ssp585––this is model list without those GCMS
modelnames_OG_trimmed = ['BCC-CSM2-MR', 'CAMS-CSM1-0', 'CESM2', 'CESM2-WACCM', 'EC-Earth3', 
                         'EC-Earth3-Veg', 'FGOALS-f3-L', 'GFDL-ESM4', 'INM-CM4-8',
                           'INM-CM5-0', 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'NorESM2-MM']

#Generic filepath to navigate to Drive folder 
fpathOG1 = '/Users/finnwimberly/Library/CloudStorage/GoogleDrive-fwimberly@middlebury.edu/My Drive/'
fpathOG2 = 'Lizz Research Stuff/Runoff-intercomparison/OGGM/lschuster/runs_2023.3/output/basins/'

In [ ]:
#Importing all runoff data, OGGM is grouped by basin
rf_ds = {}
rf_ds_monthly = {}
for basin, ID in Alpine_basins.items():
    fpath_basin = 'gcm_from_2000_bc_2000_2019/{}/'.format(ID)
    #print(f'{fpathOG1 + fpathOG2 + fpath_basin}*.nc')
    with xr.open_mfdataset(f'{fpathOG1 + fpathOG2 + fpath_basin}*.nc') as ds1:
        ds1 = ds1.runoff.load()
    rf_ds[basin] = ds1
    with xr.open_mfdataset(f'{fpathOG1 + fpathOG2 + fpath_basin}*.nc') as ds2:
        ds_monthly = ds2.runoff_monthly.load()
    rf_ds_monthly[basin] = ds_monthly

In [ ]:
#Summing individual glacier runoff into basin totals and converting kg to km^3
basin_rf_OG = {}
basin_rf_monthly_OG = {}
for basin, ID in Alpine_basins.items():
    basin_rf_OG[basin] = rf_ds[basin].sum(dim = 'rgi_id') * 1e-12
    basin_rf_monthly_OG[basin] = rf_ds_monthly[basin].sum(dim = 'rgi_id') * 1e-12

In [ ]:
#creating dict of GloPy format
basin_sums_OG = {}
basin_sums_monthly_OG = {}
for s, SSP in enumerate(SSPs):
    basin_sums_OG[SSP] = {}
    basin_sums_monthly_OG[SSP] = {}
    for b, basin in enumerate(basins):
        basin_sums_OG[SSP][basin] = basin_rf_OG[basin].sel(scenario = SSP)
        basin_sums_monthly_OG[SSP][basin] = basin_rf_monthly_OG[basin].sel(scenario = SSP)

In [ ]:
#Removing these GCMs for ALL SSPs--doing even 585 as these two are not included...
#... in Glo or Py so not only makeds OOGM easier but maintains GCM consistency in analysis
trimmed_basin_sums_OG = {}
trimmed_basin_sums_monthly_OG = {}
for s, SSP in enumerate(SSPs):
    trimmed_basin_sums_OG[SSP] = {}
    trimmed_basin_sums_monthly_OG[SSP] = {}
    for b, basin in enumerate(basins):
        trimmed_basin_sums_OG[SSP][basin] = xr.concat([basin_sums_OG[SSP][basin][0:4], basin_sums_OG[SSP][basin][5:-1]], dim='gcm')
        trimmed_basin_sums_monthly_OG[SSP][basin] = xr.concat([basin_sums_monthly_OG[SSP][basin][0:4], basin_sums_monthly_OG[SSP][basin][5:-1]], dim='gcm')

In [ ]:
#Compute multi GCM means and quartiles for OGGM
GCM_mean_OG = {}
GCM_q1_OG = {}
GCM_q3_OG = {}
for s, SSP in enumerate(SSPs):
    which_ssp = SSPs[s]
    GCM_mean_OG[which_ssp] = {}
    GCM_q1_OG[which_ssp] = {}
    GCM_q3_OG[which_ssp] = {}
    for basin in basins:
        GCM_mean_OG[which_ssp][basin] = trimmed_basin_sums_OG[which_ssp][basin].mean(dim = 'gcm')
        GCM_q1_OG[which_ssp][basin] = trimmed_basin_sums_OG[which_ssp][basin].quantile(q = 0.25, dim = 'gcm')
        GCM_q3_OG[which_ssp][basin] = trimmed_basin_sums_OG[which_ssp][basin].quantile(q = 0.75, dim = 'gcm')

In [ ]:
#Plot setup
from cycler import cycler
import matplotlib.patches as mpatches

scenarios = ['ssp126','ssp245','ssp370','ssp585']

Alpine_basins = {'CLUTHA':'5406'}

basins = ['CLUTHA']

yrs_glo = np.arange(1980,2101)
yrs_glo_dt = pd.to_datetime([str(y)for y in yrs_glo])

colors_glo =  plt.colormaps['Greens']
line_colors_glo = colors_glo(np.linspace(0.2, 0.6, num = 12))
glo_cycler = cycler(color = line_colors_glo)

colors_py =  plt.colormaps['Purples']
line_colors_py = colors_py(np.linspace(0.2, 0.6,num = 12))
py_cycler = cycler(color = line_colors_py)

colors_OG =  plt.colormaps['Blues']
line_colors_OG = colors_OG(np.linspace(0.2, 0.6,num = 12))
OG_cycler = cycler(color = line_colors_OG)

In [ ]:
fig, axs = plt.subplots(1, len(SSPs), figsize=(10, 2), sharex=True)

scenarios = ['ssp126','ssp245','ssp370','ssp585']

basin = 'CLUTHA'
basintext = 'Clutha'

for s, SSP in enumerate(scenarios):
    #OG won't plot with built-in ds.plot()
    #Trim last value as it goes to zero
    for m, model in enumerate(modelnames_OG_trimmed):
        axs[s].plot(yrs_glo_dt[20:-1], trimmed_basin_sums_OG[SSP][basin][:,0:-1].sel(gcm = modelnames_OG_trimmed[m]), color = 'dodgerblue', alpha = 0.15)
    axs[s].plot(yrs_glo_dt[20:-1], GCM_mean_OG[SSP][basin][0:-1], color = 'royalblue', linewidth = 0.9, label = 'OGGM')
    axs[s].plot(yrs_glo_dt[20:-1], GCM_q1_OG[SSP][basin][0:-1], color = 'royalblue', linewidth = 0.4)
    axs[s].plot(yrs_glo_dt[20:-1], GCM_q3_OG[SSP][basin][0:-1], color = 'royalblue', linewidth = 0.4)
    axs[s].fill_between(yrs_glo_dt[20:-1], GCM_q1_OG[SSP][basin][0:-1], GCM_q3_OG[SSP][basin][0:-1], color = 'dodgerblue', alpha = 0.5)

    for m in modelnames_glo:
        axs[s].plot(yrs_glo_dt, basin_sums_glo[SSP][basin][m], color=axs[s].set_prop_cycle(glo_cycler), alpha = 0.25)
    axs[s].plot(yrs_glo_dt, GCM_mean_glo[SSP][basin], color = 'darkgreen', linewidth = 0.9, label = 'GloGEM')
    axs[s].plot(yrs_glo_dt, GCM_q1_glo[SSP][basin], color = 'darkgreen', linewidth = 0.4)
    axs[s].plot(yrs_glo_dt, GCM_q3_glo[SSP][basin], color = 'darkgreen', linewidth = 0.4)
    axs[s].fill_between(yrs_glo_dt, GCM_q1_glo[SSP][basin], GCM_q3_glo[SSP][basin], color = 'green')
    axs[s].set(xlim=(pd.to_datetime('2000-01-01'), pd.to_datetime('2100-01-01')))

    basin_sums_py[SSP][basin].plot(hue='model', ax=axs[s], color=axs[s].set_prop_cycle(py_cycler), alpha = 0.25, add_legend=False)
    GCM_mean_py[SSP][basin].plot(hue='model', ax=axs[s], color = 'purple', linewidth = 0.9, add_legend=False, label = 'PyGEM')
    GCM_q1_py[SSP][basin].plot(hue='model', ax=axs[s], color = 'purple', linewidth = 0.4, add_legend=False)
    GCM_q3_py[SSP][basin].plot(hue='model', ax=axs[s], color = 'purple', linewidth = 0.4, add_legend=False)
    axs[s].fill_between(yrs_glo_dt[20::], GCM_q1_py[SSP][basin], GCM_q3_py[which_ssp][basin], color = 'Purple')
    axs[s].set(title = '')

    for s in range(4):  # Use a different variable name for the inner loop
        axs[s].set_xlabel('Year')
        axs[s].set_xticks([pd.to_datetime('2025'),pd.to_datetime('2050'), pd.to_datetime('2075'), pd.to_datetime('2100')], [2025, 2050, 2075, 2100])
        if s == 0:                                                                    #Setting basin labels
            axs[s].set_ylabel(r'Runoff  $[km^3]$')
        if s != 0:
            axs[s].set_ylabel('')
            axs[s].set_yticklabels('')

axs[0].legend(bbox_to_anchor=(3.15, 1.48), ncol=3)
plt.suptitle('Projected Runoff for the '+str(basintext)+' River Basin', x=0.488, y=1.36)
plt.title('SSP 126                          SSP 245                         SSP 370                          SSP 585', x=-1.3, y=1.05) 

In [ ]:
#Calculating inter-GCM variance
variance_glo = {}
variance_py = {}
variance_OG = {}
for s, SSP in enumerate(SSPs):
    variance_glo[SSP] = {}
    variance_py[SSP] = {}
    variance_OG[SSP] = {}
    for b, basin in enumerate(basins):
        variance_glo[SSP][basin] = pd.DataFrame(basin_sums_glo[SSP][basin]).var(axis=1)
        variance_py[SSP][basin] = basin_sums_py[SSP][basin].var(dim = 'model')
        variance_OG[SSP][basin] =  trimmed_basin_sums_OG[SSP][basin].var(dim = 'gcm')

In [ ]:
fig, axs = plt.subplots(1, len(SSPs), figsize=(10, 2), sharex=True)

scenarios = ['ssp126','ssp245','ssp370','ssp585']

basin = 'CLUTHA'
basintext = 'Clutha'

for s, SSP in enumerate(scenarios):
    axs[s].plot(yrs_dt, variance_glo[SSP][basin][20::], color='darkgreen')

    axs[s].plot(yrs_dt, variance_OG[SSP][basin], color = 'dodgerblue')        

    variance_py[SSP][basin].plot(hue='model', ax=axs[s], color= 'purple')
        
for s in range(4):  
    axs[s].set_xlabel('Year')
    axs[s].set_xlabel('')
    axs[s].set_xticks([pd.to_datetime('2025'),pd.to_datetime('2050'), pd.to_datetime('2075'), pd.to_datetime('2100')], [2025, 2050, 2075, 2100])
    if s == 0:                                                                    #Setting basin labels
        axs[s].set_ylabel(r'  $\sigma^2$')
    if s != 0:
        axs[s].set_ylabel('')
        axs[s].set_yticklabels('')
    
green_patch = mpatches.Patch(color='darkgreen', label='GloGEM')
purple_patch = mpatches.Patch(color='purple', label='PyGEM') 
blue_patch = mpatches.Patch(color='royalblue', label='OGGM')
axs[0].legend(handles=[green_patch, purple_patch, blue_patch],bbox_to_anchor=(3.15, 1.4), ncol=3)
plt.suptitle('Inter-Annual Variance for Projected Runoff of the '+str(basintext)+' River Basin', x=0.488, y=1.28)
plt.title('SSP 126                          SSP 245                         SSP 370                          SSP 585', x=-1.3, y=1) 

In [ ]:
#Calculating interannual variance
rolling_years = 15
yrs = np.arange(2000,2101)
yrs_dt = pd.to_datetime([str(y)for y in yrs])

annual_variance_glo = {}
annual_variance_py = {}
annual_variance_OG = {}
for s, SSP in enumerate(SSPs):
    annual_variance_glo[SSP] = {}
    annual_variance_py[SSP] = {}
    annual_variance_OG[SSP] = {}
    for b, basin in enumerate(basins):
        #annual_variance_glo[SSP][basin] = pd.DataFrame(basin_sums_glo[SSP][basin]).rolling(window= rolling_years, min_periods=10, center = True).var(axis=0)
        annual_variance_py[SSP][basin] = basin_sums_py[SSP][basin].rolling(time=rolling_years, min_periods=10, center = True).var()
        annual_variance_OG[SSP][basin] =  trimmed_basin_sums_OG[SSP][basin][0::,0:-1].rolling(time=rolling_years, min_periods=10, center = True).var()
        annual_variance_glo[SSP][basin] = {}
        for m in modelnames_glo:
            annual_variance_glo[SSP][basin][m]= pd.DataFrame(basin_sums_glo[which_ssp][basin][m]).rolling(window=rolling_years, min_periods=10, center = True).var()

In [ ]:
mean_variance_glo = {}
for s, SSP in enumerate(SSPs):
    mean_variance_glo[SSP] = {}
    for b, basin in enumerate(basins):
        mean_variance_glo[SSP][basin] = GCM_mean_glo[SSP][basin].rolling(window=rolling_years, min_periods=10, center = True).var()

In [ ]:
fig, axs = plt.subplots(1, len(SSPs), figsize=(10, 2), sharex=True)

scenarios = ['ssp126','ssp245','ssp370','ssp585']

basin = 'CLUTHA'
basintext = 'Clutha'

for s, SSP in enumerate(scenarios):
    for m in modelnames_glo:               #Plotting Rf inter-annual Variance
        axs[s].plot(yrs_dt, annual_variance_glo[SSP][basin][m][20::], color=axs[s].set_prop_cycle(glo_cycler), alpha = 0.25)
        
    for m, model in enumerate(modelnames_OG_trimmed):
        axs[s].plot(yrs_dt[0:-1], annual_variance_OG[SSP][basin].sel(gcm = modelnames_OG_trimmed[m]), color = 'dodgerblue', alpha = 0.15)        

        annual_variance_py[SSP][basin].plot(hue='model', ax=axs[s], color=axs[s].set_prop_cycle(py_cycler), alpha = 0.55, add_legend=False)
        
        axs[s].plot(yrs_dt, annual_variance_py[SSP][basin].mean(dim = 'model'), color = 'purple', label = 'PyGEM')
        axs[s].plot(yrs_dt, mean_variance_glo[SSP][basin][20::], color = 'darkgreen', label = 'GloGEM')
        axs[s].plot(yrs_dt[0:-1], annual_variance_OG[SSP][basin].mean(dim = 'gcm'), color = 'royalblue', label = 'OGGM')

for s in range(4):  
    axs[s].set_xlabel('Year')
    axs[s].set_xlabel('')
    axs[s].set_xticks([pd.to_datetime('2025'),pd.to_datetime('2050'), pd.to_datetime('2075'), pd.to_datetime('2100')], [2025, 2050, 2075, 2100])
    if s == 0:                                                                    #Setting basin labels
        axs[s].set_ylabel(r'  $\sigma^2$')
    if s != 0:
        axs[s].set_ylabel('')
        axs[s].set_yticklabels('')
    
green_patch = mpatches.Patch(color='darkgreen', label='GloGEM')
purple_patch = mpatches.Patch(color='purple', label='PyGEM') 
blue_patch = mpatches.Patch(color='royalblue', label='OGGM')
axs[0].legend(handles=[green_patch, purple_patch, blue_patch],bbox_to_anchor=(3.15, 1.4), ncol=3)
plt.suptitle('Inter-Annual Variance for Projected Runoff of the '+str(basintext)+' River Basin', x=0.488, y=1.28)
plt.title('SSP 126                          SSP 245                         SSP 370                          SSP 585', x=-1.3, y=1) 

### CSV File Read Out

In [ ]:
## set up filename to reflect what you're writing out, possibly in a nested loop
modelnames_all = ['BCC-CSM2-MR', 'CESM2', 'CESM2-WACCM', 'EC-Earth3', 'EC-Earth3-Veg', 'FGOALS-f3-L', 'GFDL-ESM4', 
                  'INM-CM4-8', 'INM-CM5-0', 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'NorESM2-MM']

scenarios = ['ssp126','ssp245','ssp370','ssp585']
basins = ['CLUTHA']

In [ ]:
#Turning OGGM 2d time array into 1d df
df = {}
df_list = {}
for s, SSP in enumerate(SSPs):
    df[SSP] = {}
    df_list[SSP] = {}
    for b, basin in enumerate(basins):
        df[SSP][basin] = {}
        df_list[SSP][basin] = {}
        for m, model in enumerate(modelnames_all):
            df[SSP][basin][model] = {}
            df_list[SSP][basin][model] = []
            for n in range(101):
                df[SSP][basin][model][2000+n] = pd.DataFrame(trimmed_basin_sums_monthly_OG[SSP][basin]. sel(gcm = model, time = 2000+n))
                df_list[SSP][basin][model].append(df[SSP][basin][model][2000+n])
                
basin_sums_monthly_OG_df = {}
for s, SSP in enumerate(SSPs):
     basin_sums_monthly_OG_df[SSP] = {}
     for b, basin in enumerate(basins):
         basin_sums_monthly_OG_df[SSP][basin] = {}
         for m, model in enumerate(modelnames_all):
             basin_sums_monthly_OG_df[SSP][basin][model] = pd.concat(df_list[SSP][basin][model], axis = 0)

In [ ]:
#Using the GloGEM datetime index
indices = basin_sums_monthly_glo['ssp126'][basins[0]]['BCC-CSM2-MR'][240::].index

#Creating dataframes of SSP, basin, and GCM containing all 3 global glacier models
out_df = {}
for s, SSP in enumerate(scenarios):
    out_df[SSP] = {}
    for b, basin in enumerate(basins):
        out_df[SSP][basin] = {}
        for m, model in enumerate(modelnames_all):
            glo_values = basin_sums_monthly_glo[SSP][basin][model][240::].values.flatten()
            pygem_values = pd.DataFrame(basin_sums_monthly_py[SSP][basin].sel(model=m + 1)).values.flatten()
            oggm_values = basin_sums_monthly_OG_df[SSP][basin][model].values.flatten()

            out_df[SSP][basin][model] = pd.DataFrame(
                {
                    'GloGEM': glo_values,
                    'PyGEM': pygem_values,
                    'OGGM': oggm_values,
                },
                index=indices
            )

In [ ]:
# Define the directory to save the CSV files
output_dir = '/Users/finnwimberly/Desktop/Lizz Research/CSV Outputs/Runoff/RGI 18/'

for SSP in out_df:
    for basin in out_df[SSP]:
        for GCM in out_df[SSP][basin]:
            fname = f"runoff_{GCM}_{SSP}_{basin}.csv"

            # Define the full path of the output file
            output_path = os.path.join(output_dir, fname)

            # Save the DataFrame as CSV
            out_df[SSP][basin][GCM].to_csv(output_path, header=True, index=True)